In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project = "SPAT"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [ ]:
from quick_pp.lithology.sand_silt_clay import SandSiltClay
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import *
from quick_pp.plotter import *
from quick_pp.permeability import *
from quick_pp.ressum import *
from quick_pp.rock_type import rock_typing
from quick_pp.plotter import plotly_log
from quick_pp.utils import zone_flagging, min_max_line

In [ ]:
# Load data
well_name = 'SPAT-A005'
well = project.get_well(well_name)
well_data = well.data

min_depth = 4748
max_depth = well_data.DEPTH.max()
well_data = well_data[(well_data.DEPTH >= min_depth) & (well_data.DEPTH <= max_depth)].copy()

# Clean up data
well_data = badhole_flag(well_data)
well_data = mask_outside_threshold(well_data, fill=True)

# Initialize lithology model
args = {
    'litho_model': 'ssc',
    # 'dry_clay_point': (.3, 2.7),
    'silt_line_angle': 116,
    'wet_clay_point': (0.43, 2.6),
    'sw_water_salinity': 5000,
    'sw_m': 1.8,
    'hc_corr_angle': 50,
    'hc_buffer': 0.1,
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    ),
}
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, vclb, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    well_data['NPHI'], well_data['RHOB'], model='kuttan_modified', normalize= True
)
args.update(ssc_model.__dict__)
well.update_config(args)  # Save lithology model to well

# Implement hydrocarbon correction
nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
    well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'],
    dry_sand_point=args['dry_sand_point'],
    dry_clay_point=args['dry_clay_point'],
    corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
)

# Estimate lithology
ssc_model = SandSiltClay(**args)
vsand, vsilt, vcld, vclb, (nphi_max_line, rhob_max_line) = ssc_model.estimate_lithology(
    nphihc, rhobhc, model='kuttan_modified', normalize= True
)
vclw = vcld + vclb

# Estimate porosity
phit = neu_den_xplot_poro(
    nphihc, rhobhc, model='ssc', reservoir=True,
    dry_sand_point=args['dry_sand_point'],
    dry_silt_point=args['dry_silt_point'],
    dry_clay_point=args['dry_clay_point'],
)

# PHID needs unnormalized lithology
vsand_un, vsilt_un, vcld_un, vclb_un, (nphi_max_line, rhob_max_line) = SandSiltClay(**args).estimate_lithology(
    nphihc, rhobhc, model='kuttan_modified', normalize= False
)
rho_ma = rho_matrix(vsand_un, vsilt_un, vcld_un)
phid = density_porosity(rhobhc, rho_ma)
phie = phit - vclb

# Estimate water saturation
temp_grad = estimate_temperature_gradient(well_data['TVDSS'], 'metric')
water_salinity = args['sw_water_salinity']
rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)

b = estimate_b_waxman_smits(temp_grad, rw_salinity)
qv = estimate_qv(vcld, phit, cec_clay=.1)

rt_water = estimate_rt_water_trend(well_data['RT'])
# rw = estimate_rw_archie(phit, rt_water)
rw = estimate_rw_waxman_smits(phit, rt_water, B=b, Qv=qv)

# TODO: Implement m estimation/ based on zonation
m = 1.85

swt = waxman_smits_saturation(well_data['RT'], rw_salinity, phit, qv, m=args['sw_m'])
swt_a = archie_saturation(well_data['RT'], rw_salinity, phit, 1, 2, 2)

# Estimate permeability
perm = choo_permeability(vclw, vsilt, phit)
# constant = vclb**1.75
# Swirr = constant / phit
# perm_kc = kozeny_carman_permeability(phit, Swirr)
# perm_t = timur_permeability(phit, Swirr)
# perm_c = coates_permeability(phit, Swirr)
# perm_tx = tixier_permeability(phit, Swirr)

# Estimate rock types
rock_flag = rock_typing(vclw)

# Update ZONES
well_data = zone_flagging(well_data)
zones = well_data.ZONES

# Estimate reservoir summary
ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm, zones, cutoffs=args['ressum_cutoffs'])


# Update data in the project
well_data['NPHI_HC'] = nphihc
well_data['RHOB_HC'] = rhobhc
well_data['HC_FLAG'] = hc_flag
well_data['VSAND'] = vsand
well_data['VSILT'] = vsilt
well_data['VCLB'] = vclb
well_data['VCLD'] = vcld
well_data['VCLW'] = vclw
# well_data['PHIT'] = phit
well_data['PHIE'] = phit  # Temporary
well_data['PHID'] = phid
well_data['RW'] = rw
well_data['B'] = b
well_data['Qv'] = qv
well_data['M'] = m
# well_data['SWT'] = swt
well_data['SWE'] = swt  # Temporary
well_data['PERM'] = perm
well_data['ROCK_FLAG'] = rock_flag
well_data['VHC'] = well_data['PHIT'] * (1 - well_data['SWT'])

In [ ]:
fig, (ax2, ax1) = plt.subplots(nrows=2, figsize=(20, 3), sharex=True)

ax1.plot(well_data.DEPTH, well_data.NPHI, label='NPHI')
ax1.plot(well_data.DEPTH, nphi_max_line, label='NPHI_line')
ax1.set_ylim(.45, -.15)
ax1.legend()

ax2.plot(well_data.DEPTH, well_data.RHOB, label='RHOB')
ax2.plot(well_data.DEPTH, rhob_max_line, label='RHOB_line')
ax2.set_ylim(1.85, 2.85)
ax2.legend()

In [ ]:
copy_args = args.copy()
copy_args.pop('silt_line_angle')
print(copy_args)
neutron_density_xplot(well_data['NPHI'], well_data['RHOB'], **copy_args)

In [ ]:
neutron_density_xplot(nphihc, rhobhc, **copy_args)

In [ ]:
well_data[['DEPTH', 'TVDSS']]

In [ ]:
# Debug water saturation
test = well_data.copy()
water_salinity = 5000
m = 1.8

temp_grad = estimate_temperature_gradient(test['TVDSS'], 'imperial')
rw_salinity = estimate_rw_temperature_salinity(temp_grad, water_salinity)
rw_archie = estimate_rw_archie(test['PHIT'], test['RT'], m=m)

b_sal = estimate_b_waxman_smits(temp_grad, rw_salinity)
b_archie = estimate_b_waxman_smits(temp_grad, rw_archie)

qv = estimate_qv(vcld, phit, cec_clay=.1)

rt_trend = estimate_rt_water_trend(test['RT'], alpha=.1)
rw_ws = estimate_rw_waxman_smits(test['PHIT'], rt_trend, m=m, B=b_archie, Qv=qv)

swt_sal = waxman_smits_saturation(test['RT'], rw_salinity, test['PHIT'], qv)
swt_ws = waxman_smits_saturation(test['RT'], rw_ws, test['PHIT'], qv)

# Plotting
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, sharex=True, figsize=(20, 5))

ax1.plot(test.DEPTH, qv, label='Qv')
ax1.set_ylim(0, 10)
ax1.legend()

ax2.plot(test.DEPTH, b_sal, label='b_sal')
ax2.plot(test.DEPTH, b_archie, label='b_archie')
ax2.legend()

ax3.plot(test.DEPTH, test['RT'], label='RT')
ax3.plot(test.DEPTH, rt_trend, label='RT water trend')
ax3.set_yscale('log')
ax3.legend()

ax4.plot(test.DEPTH, rw_salinity, label='rw_salinity')
ax4.plot(test.DEPTH, rw_archie, label='rw_archie')
ax4.plot(test.DEPTH, rw_ws, label='rw_ws')
ax4.set_yscale('log')
ax4.legend()

ax5.plot(test.DEPTH, swt_sal, label='SWT_salinity')
ax5.plot(test.DEPTH, swt_ws, label='SWT_ws')
ax5.plot(test.DEPTH, test['SWT'], label='SWT')
ax5.legend()

In [ ]:
# Plot the results
well_data.drop(columns=['VOIL', 'VGAS'], errors='ignore', inplace=True)
fig = plotly_log(well_data, depth_uom=well.depth_uom)
fig.show()

In [ ]:
from quick_pp.qaqc import quick_qc

test, summary_df = quick_qc(well_data)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(20, 5), sharex=True)

ax1.plot(test['DEPTH'], test.VCLW, label='VSHALE')
ax1.plot(test['DEPTH'], test.VSH_GR, label='VSH_GR')
ax1.legend()

ax2.plot(test['DEPTH'], test.PHIT, label='PHIT')
ax2.legend()

ax3.plot(test['DEPTH'], test.SWT, label='SWT')
ax3.plot(test['DEPTH'], test.SAND_FLAG, label='SAND_FLAG')
ax3.legend()

ax4.plot(test['DEPTH'], test['QC_FLAG'], label='QC_FLAG')
ax4.legend()

summary_df.head(10)

In [ ]:
# # Save the results to the well
# well.update_data(well_data)
# well.update_ressum(ressum_df)
# project.save_well(well)

In [ ]:
# project.save()